In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path

from dfm_models.utils.io import download_COOPs, download_nwis
from dfm_models.utils.openDA import writeNoosTs, createNoosConfigFile

In [ ]:
project = Path("/mnt/c/Users/rdchlclj/Projects/MR_D3D_model")
stations = pd.read_excel(project / "Delft3D/observations/stations.xlsx", dtype={"station_id": str})

In [ ]:
output = Path.cwd() / "dumstochObserver"
#output.rmdir()
output.mkdir()
product = "hourly_height"
datum = "MSL"
form = "csv"
begin_date = "20160101"
end_date = "20160102"
unit = "waterlevel"

dt = pd.to_timedelta(6, "H")

for i, row in stations.iterrows():

    station_id = row.station_id
    obsFile_id = row.obsFile_id
    x = row.longitude
    y = row.latitude
    provider = row.provider

    if provider == "NOAA":
        data = download_COOPs(
            product, obsFile_id, station_id, datum, begin_date, end_date
        )
        
        data.dropna(inplace=True)
        #data.index = data.index + dt

        fn = output / f"{obsFile_id}"
        writeNoosTs(data, obsFile_id, x, y, unit, fn)

fn = output / "noosObservations.xml"
stationNames = stations[stations.provider == "NOAA"]["obsFile_id"].values
createNoosConfigFile(0.05, stationNames, fn)